In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits

import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]


In [ ]:


data = fits.getdata("image.fits")

In [ ]:
data

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower',)
plt.colorbar();
plt.savefig('/Users/labuser/Desktop/data.png')

# Background subrtaction

In [ ]:
#measure a sparially varying background on the image
bkg = sep.Background(data)
#returns a background object that hold info on the spatially varying background noise lvl

In [ ]:
#get a "global" mean and noise of the image background
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
#evaluate background as 2D array, same size as the original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg)

In [ ]:
#show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('/Users/labuser/Desktop/bkg_image.png')

In [ ]:
#evaluate the background nois as a 2D array, same as the original image
bkg_rms = bkg.rms()


In [ ]:
#show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('/Users/labuser/Desktop/bkg-rms.png')

In [ ]:
#subtract the background
data_sub = data - bkg
#can also subtrct the background for the data array in-place by doing bkg.subform(data)

# Object detection

In [ ]:
#now that we subtracted the background we can run object detection on the background-subtracted data

#background noise is pretty flat, so we set detection threshold to be cont value of 1.5sigma where sigma=background RMS

objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
#sep.extract has many options for controlling detection threshold, pixel mask, filter 
#object is a numpy array with many fields

#how many objects were detected?
len(objects)

In [ ]:
#objects['x'] and objects['y'] will give the centroid coords for the objects
#we'll plot over the object coordinates to see where they are

#plot background-subtracted image

from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.savefig('/Users/labuser/Desktop/data_sub.png')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), 
                width=6*objects['a'][i], 
                height=6*objects['b'][i], 
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

In [ ]:
#objects has many other fields giving info such as second moments, peak pixel position and values. 
#see available fields
objects.dtype.names

# Aperture photometry

In [ ]:
#finally we preform a cirular aperture photometry w/ a 3 pixel radius at the locations of the objects

flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

#flux, fluxerr and flag are all 1D arrays with one entry per object

In [ ]:
#show the first 10 object results
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))